In [1]:
import os
os.environ["CUDA_VISIBLE_DEVIES"]="0,5,6,7"
import torch
# 设置可见的GPU
# torch.cuda.set_device("cuda:7")


In [2]:
# 目录设置为 src
import sys
sys.path.append("src")

from generate import BasicGenerator

generator = BasicGenerator("meta-llama/Llama-2-7b-chat-hf")


/home/nsf/anaconda3/envs/dragin/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:generate:Loading model from meta-llama/Llama-2-7b-chat-hf
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.06s/it]
Using pad_token, but it is not set yet.


In [4]:
import json
import os
data_path = 'data/2wikimultihopqa/'
with open(os.path.join(data_path, 'dev_10.json'), 'r') as fin:
    js = json.load(fin)

In [6]:
js[2]

{'_id': 'e2a3bf2a0bdd11eba7f7acde48001122',
 'type': 'compositional',
 'question': "When did John V, Prince Of Anhalt-Zerbst's father die?",
 'context': [['Karl I, Prince of Anhalt-Zerbst',
   ['Karl I of Anhalt- Zerbst( 17 November 1534 in Dessau – 4 May 1561 in Zerbst), was a German prince of the House of Ascania and ruler of the principality of Anhalt- Zerbst.',
    'He was the eldest son of John V, Prince of Anhalt- Zerbst, by his wife Margaret, daughter of Joachim I Nestor, Elector of Brandenburg.']],
  ['Bernhard VII, Prince of Anhalt-Zerbst',
   ['Bernhard VII of Anhalt- Zerbst( 17 March 1540 – 1 March 1570), was a German prince of the House of Ascania and ruler of the principality of Anhalt- Zerbst.',
    'He was born and died in Dessau, and was the third and youngest son of John V, Prince of Anhalt- Zerbst by his wife Margaret, daughter of Joachim I Nestor, Elector of Brandenburg.']],
  ['Ernest I, Prince of Anhalt-Dessau',
   ['Ernest I, Prince of Anhalt-Dessau (died Dessau, 

In [8]:
from src.retriever import BM25
from transformers import AutoTokenizer

tokenizer =  AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
retriever = BM25(
                    tokenizer = tokenizer,
                    index_name = "wiki",
                    engine = "elasticsearch",
                )

INFO:root:Activating Elasticsearch....
INFO:root:Elastic Search Credentials: {'hostname': 'localhost', 'index_name': 'wiki', 'keys': {'title': 'title', 'body': 'txt'}, 'timeout': 100, 'retry_on_timeout': True, 'maxsize': 24, 'number_of_shards': 1, 'language': 'english'}


In [9]:
query = """
Ernest I, Prince of Anhalt-Dessau, father of John V, Prince of Anhalt-Zerbst, died in 1602.
"""

_docs_ids, docs1 = retriever.retrieve(
                queries = [query],
                topk = 5,
                max_query_length = 64,
            )

In [36]:
for doc in docs1[0]:
    # print(doc)
    # print(80*'-')
    print('\n'.join([doc[i:i+120] for i in range(0, len(doc), 120)]))
    print(80*'-')

John V, Prince of Anhalt-Zerbst John V of Anhalt-Zerbst (Dessau, 4 September 1504 – Zerbst, 4 February 1551), was a Germ
an prince of the House of Ascania and ruler of the principality of Anhalt-Dessau. From 1544, he assumed rule of the re-c
reated principality of Anhalt-Zerbst. John was the second (but eldest surviving) son of Ernest I, Prince of Anhalt-Dessa
u, by his wife Margarete, daughter of Henry I, Duke of Münsterberg-Oels, and granddaughter of George of Poděbrady, King 
of Bohemia. Upon the death of his father in 1516, John and his brothers George III and Joachim I inherited Anhalt-Dessau
 as co-rulers
--------------------------------------------------------------------------------
Bernhard VII, Prince of Anhalt-Zerbst Bernhard VII of Anhalt-Zerbst (17 March 1540 – 1 March 1570), was a German prince 
of the House of Ascania and ruler of the principality of Anhalt-Zerbst. He was born and died in Dessau, and was the thir
d and youngest son of John V, Prince of Anhalt-Zerbst by h

In [79]:
query = """
Date of death of John V, Prince of Anhalt-Zerbst's father.
"""

_docs_ids, docs2 = retriever.retrieve(
                queries = [query],
                topk = 5,
                max_query_length = 64,
            )

In [11]:
for doc in docs2[0]:
    # print(doc)
    # print(80*'-')
    print('\n'.join([doc[i:i+120] for i in range(0, len(doc), 120)]))
    print(80*'-')

NameError: name 'docs2' is not defined

In [10]:
cot = """
Question: When did the director of film Hypocrite (Film) die?
Answer: The film Hypocrite was directed by Miguel Morayta. Miguel Morayta died on 19 June 2013. So the answer is 19 June 2013.
Question: Are both Kurram Garhi and Trojkrsti located in the same country?
Answer: Kurram Garhi is located in the country of Pakistan. Trojkrsti is located in the country of Republic of Macedonia. Thus, they are not in the same country. So the answer is no.
Question: Do director of film Coolie No. 1 (1995 Film) and director of film The Sensational Trial have the same nationality?
Answer: Coolie No. 1 (1995 film) was directed by David Dhawan. The Sensational Trial was directed by Karl Freund. David Dhawan's nationality is India. Karl Freund's nationality is Germany. Thus, they do not have the same nationality. So the answer is no.
Question: Who is Boraqchin (Wife Of Ögedei)'s father-in-law?
Answer: Boraqchin is married to Ögedei Khan. Ögedei Khan's father is Genghis Khan. Thus, Boraqchin's father-in-law is Genghis Khan. So the answer is Genghis Khan.
Question: Who was born first out of Martin Hodge and Ivania Martinich?
Answer: Martin Hodge was born on 4 February 1959. Ivania Martinich was born on 25 July 1995. Thus, Martin Hodge was born first. So the answer is Martin Hodge.
Question: When did the director of film Laughter In Hell die?
Answer: The film Laughter In Hell was directed by Edward L. Cahn. Edward L. Cahn died on August 25, 1963. So the answer is August 25, 1963.
"""
# prompt = "Context: "
# prompt += docs1[0][0]
# prompt += docs2[0][0]
prompt = "\nIdentify the essential information required to answer the question, listing them as 1. 2. \n"
prompt += "Question is: " + js[2]['question']
prompt += "\nmissing: "
res, _, _ = generator.generate(prompt, max_length=128)

In [41]:
print(prompt)

Question: When did John V, Prince Of Anhalt-Zerbst's father die?
Identify the essential key information required to answer the question, listing them as 1. 2. 

missing: 


In [12]:
print(res)



1. Name of John V, Prince Of Anhalt-Zerbst's father
2. Date of death of John V, Prince Of Anhalt-Zerbst's father

Answer:
John V, Prince Of Anhalt-Zerbst's father, Christian August, died on 10th August 1747.</s>


In [79]:
prompt = "Name of John V, Prince Of Anhalt-Zerbst's father"

prompt += "Check the answer for logical errors. If there are any, output correction suggestions; otherwise, output \"The answer is complete\".\n Here is the answer:\n"
prompt += res + "\n"
prompt += "Provide a brief analysis. Correction suggestions:"

rep, _, _ = generator.generate(prompt, max_length=128)

In [80]:
rep

'\nBlind Shaft (2007) was released before The Mask Of Fu Manchu (1965).\nThe answer is complete.</s>'

In [41]:
res = js[0]['question']
# 删除()内的内容
import re
res = re.sub(r'\([^)]*\)', '', res)
res
prompt = "Analyze following question and answer."
prompt += "Please format the output as 1.xxx2.xxx. "
prompt += "Question: " + res + "\n"
prompt += "Answer: "
# prompt += "1. The director of film Polish-Russian War is Xawery Żuławski. "
# prompt += "Xawery Żuławski's mother is Maria Kuncewicz. "
# prompt += "So the answer is Maria Kuncewicz. "
# prompt += "\nThe next steps are: "
rep, _, _ = generator.generate(prompt, max_length=128)

In [11]:
prompt

"Context: John V, Prince of Anhalt-Zerbst John V of Anhalt-Zerbst (Dessau, 4 September 1504 – Zerbst, 4 February 1551), was a German prince of the House of Ascania and ruler of the principality of Anhalt-Dessau. From 1544, he assumed rule of the re-created principality of Anhalt-Zerbst. John was the second (but eldest surviving) son of Ernest I, Prince of Anhalt-Dessau, by his wife Margarete, daughter of Henry I, Duke of Münsterberg-Oels, and granddaughter of George of Poděbrady, King of Bohemia. Upon the death of his father in 1516, John and his brothers George III and Joachim I inherited Anhalt-Dessau as co-rulers\n Answer the following question by reasoning step-by-step. Question: When did John V, Prince Of Anhalt-Zerbst's father die?\nAnswer:"

In [42]:
rep

'1. The mother of the director of the film Polish-Russian War is not known.\n2. The director of the film Polish-Russian War is not a real person, but a fictional character created for the purpose of the story.</s>'

In [45]:
from torch import exp


res = docs1[0][0]
prompt = "Context: " + res + "\n"
prompt = "Provide only the answer to sub-question 1: Name of John V, Prince Of Anhalt-Zerbst's father. \n"
prompt += "Answer to sub-question 1: "
# prompt += "Answer: "
# prompt += "John V, Prince of Anhalt-Zerbst's father is Ernest I, Prince of Anhalt-Dessau. Ernest I, Prince of Anhalt-Dessau died in 1602. So the answer is 1602."
# prompt += ""
# prompt += "\nInformation requirements: "
rep, _, _ = generator.generate(prompt, max_length=128)

new_text, tokens, attns, logprobs, entropies = generator.generate_attn(
                prompt,
                128,
                # self.attention_solver,
                use_entropy =False,
                use_logprob = True
            )
logprobs = torch.tensor(logprobs)
print(exp(logprobs))




print(rep)
# prompt += rep.strip() + "\n"
# prompt += "Provide the confidence level of the answer as a score between 0 and 100. Confidence: "
# rep, _, _ = generator.generate(prompt, max_length=128)
# print(rep)

# new_text, tokens, attns, logprobs, entropies = generator.generate_attn(
#                 prompt,
#                 128,
#                 # self.attention_solver,
#                 use_entropy =False,
#                 use_logprob = True
#             )
# logprobs = torch.tensor(logprobs)
# print(exp(logprobs))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.8776,
        1.0000, 1.0000, 1.0000], dtype=torch.float64)

John V, Prince Of Anhalt-Zerbst's father was Christian August, Prince Of Anhalt-Zerbst.</s>


In [36]:
new_text, tokens, attns, logprobs, entropies = generator.generate_attn(
                prompt,
                128,
                # self.attention_solver,
                use_entropy =True,
                use_logprob = False
            )
print(entropies)

[0.671502153078715]


In [17]:
print(prompt)


Please answer the following questions and provide the confidence level of each answer using "Certain" or "Uncertain". Format your response as follows:

1. Question 1
Answer: [Your answer]
Confidence: Certain/Uncertain

2. Question 2
Answer: [Your answer]
Confidence: Certain/Uncertain

Ensure each answer is followed by the corresponding confidence level.
Name of John V, Prince Of Anhalt-Zerbst's father. 



In [44]:
res = js[2]['question']
prompt = "John V, Prince of Anhalt-Zerbst's father is Ernest I, Prince of Anhalt-Dessau. "
# prompt += "\nInformation requirements: "
prompt += "Date of death of John V, Prince of Anhalt-Zerbst's father. "
prompt += "Combine the above content into one sentence as a search keyword. "
prompt += "Search keyword: "
rep, _, _ = generator.generate(prompt, max_length=64)


In [102]:
rep

"John V, Prince of Anhalt-Zerbst's father Ernest I, Prince of Anhalt-Dessau died in 1692.</s>"

In [96]:
import spacy
nlp = spacy.load("en_core_web_sm")

missing = rep
missing = [sent.text for sent in nlp(missing).sents]

# missing = [m.split(f"{i}.")[1] for i, m in enumerate(missing) if len(m.strip()) > 0]

In [97]:
for m in missing:
    print(m)
    print(80*'-')



Answer: The director of film Wearing Velvet Slippers Under A Golden Umbrella won the Best Director award at the 2019 Cannes Film Festival.


--------------------------------------------------------------------------------
Missing:
1.
--------------------------------------------------------------------------------
Information on the film Wearing Velvet Slippers Under A Golden Umbre
--------------------------------------------------------------------------------


In [20]:
import re
info = []
matched_sentences = re.findall(r'\d+\.\s*(.+)', missing)

# 存储提取到的句子
extracted_sentences = [sentence.strip() for sentence in matched_sentences]

print(extracted_sentences)

['The name of the director of the film Polish-Russian War.', 'The name of the mother of the director.']


In [19]:
matched_sentences

[]